In [1]:
import nbsetup as nb
import numpy as np
import ergodicpy as ep
import seaborn as sns
import pandas as pd
from simulations.automata.models import CA1DEnsemble
import warnings
warnings.filterwarnings('ignore')

In [2]:
def cas_for_p(rule=226, cells=100, ensembles=20, steps=100, increments=100):
    cas = {}
    ps = np.linspace(0.0, 1.0, increments)
    for p in ps:
        nb.cp(p)
        # generate ensembles for each increment of p
        ca = CA1DEnsemble(rule=rule, cells=cells, count=ensembles, init='sparse',p=p)
        ca.run(steps)
        ca.analyse()
        cas[p] = ca
    return cas

def plot_complexity(cas, ignore_unstables=True, xlim=(0.1, 0.9)):
    dataframe = pd.concat([c.get_analysis('complexity (2)') for c in cas.values()], ignore_index=True)
    dataframe['p'] = cas.keys()
    if ignore_unstables:
        dataframe = dataframe.drop(columns=['Avg Cell Entropy','Last Cell Entropy'])
    h = sns.lineplot(data=dataframe.melt(id_vars='p'), x='p', y='value', hue='variable', palette='mako')
    sns.lineplot(y=dataframe.drop(columns='p').sum(axis=1), x=dataframe['p'], label="Sum")
    h.set(xlim=xlim, ylim=(0, None))
    return dataframe

# Initial conditions

One of the hypothesis of ergodic complexity, is that complex cases are so tricky to comes to grips with because they are particularly sensitive to initial conditions. Put in the terms of statistical / termodyanmics terminology, given differences in micro-variables (like initial positions of particles) even when macro-variables (like total number of particles) is consistent, the end results of the system does not converge across ensembles.

Scanning across the results of _p_ (p is the % of cells that start as a 1 rather than a 0), we find that for the example disordered case (rule 30) and the mechanistic simple case (rule 0) this holds as we expect in the thermodynamic case.

In [3]:
cas30 = cas_for_p(rule=30, increments=100)
cas30_complexities = plot_complexity(cas30)

0.0


KeyboardInterrupt: 

In [ ]:
cas0 = cas_for_p(rule=0, increments=50)
cas0_complexities = plot_complexity(cas0)

## Complex case

However, what we find, is for the ergodically complex case (rule 226) we see that the complexity is high with varying initial conditions.

In [ ]:
cas226 = cas_for_p(rule=226, increments=100)
cas226_complexities = plot_complexity(cas226)

0.25252525252525254


# Tipping point

More specifically, what we find is we see near _p=0.5_ is a spike in complexity, which often indicates a tipping point. More interestingly, we see that the diagonal complexity in particular swaps at this point. Looking at the results we find the following.

In [ ]:
cas226